In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
load_dotenv("/home/kevin/projects/rag_langchain/.env")
def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGSMITH_API_KEY")
        if result is None or result.strip() == "":
            print(
                "LangChain API Key가 설정되지 않았습니다."
            )
            return
        os.environ["LANGCHAIN_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")

langsmith(project_name="lcel_dev")

LangSmith 추적을 시작합니다.
[프로젝트명]
lcel_dev


In [3]:
llm= ChatOpenAI(model_name='gpt-4o',temperature=0.1)

# RunnableParallel

In [4]:
from langchain_core.runnables import RunnableParallel

In [5]:
chain1 =(
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    |llm
    |StrOutputParser()
)
chain2 = (
    PromptTemplate.from_template("{country}의 면적은 얼마야?")
    |llm
    |StrOutputParser()
)

combined = RunnableParallel(captial=chain1, area=chain2)

In [6]:
chain1.invoke({"country":"스페인"})

'스페인의 수도는 마드리드입니다. 마드리드는 스페인의 중심부에 위치해 있으며, 정치, 경제, 문화의 중심지로서 중요한 역할을 하고 있습니다.'

In [7]:
chain2.invoke({"country":"스페인"})

'스페인의 면적은 약 505,990 평방 킬로미터(약 195,360 평방 마일)입니다. 이는 스페인을 유럽에서 네 번째로 큰 국가로 만듭니다.'

In [8]:
combined.invoke({"country":"스페인"})

{'captial': '스페인의 수도는 마드리드입니다. 마드리드는 스페인의 정치, 경제, 문화의 중심지로서 많은 역사적 건축물과 현대적인 시설을 갖추고 있습니다.',
 'area': '스페인의 면적은 약 505,990 평방 킬로미터(약 195,360 평방 마일)입니다. 이는 스페인을 유럽에서 네 번째로 큰 국가로 만듭니다.'}

In [9]:
chain1.batch([{"country":"스페인"},{"country":"포르투갈"}])

['스페인의 수도는 마드리드입니다. 마드리드는 스페인의 정치, 경제, 문화의 중심지로서 중요한 역할을 하고 있습니다.',
 '포르투갈의 수도는 리스본(Lisbon)입니다. 리스본은 포르투갈의 가장 큰 도시이자 주요 경제, 문화, 정치 중심지입니다.']

In [10]:
chain2.batch([{"country":"스페인"},{"country":"포르투갈"}])

['스페인의 면적은 약 505,990 평방 킬로미터(약 195,360 평방 마일)입니다. 이는 스페인을 유럽에서 네 번째로 큰 국가로 만듭니다.',
 '포르투갈의 면적은 약 92,090 평방 킬로미터(약 35,556 평방 마일)입니다. 이는 포르투갈 본토와 아조레스 제도 및 마데이라 제도를 포함한 전체 면적을 의미합니다.']

In [11]:
combined.batch([{"country":"스페인"},{"country":"포르투갈"}])


[{'captial': '스페인의 수도는 마드리드입니다. 마드리드는 스페인의 중심부에 위치해 있으며, 정치, 경제, 문화의 중심지로서 중요한 역할을 하고 있습니다.',
  'area': '스페인의 면적은 약 505,990 평방 킬로미터(약 195,360 평방 마일)입니다. 이는 스페인을 유럽에서 네 번째로 큰 국가로 만듭니다.'},
 {'captial': '포르투갈의 수도는 리스본(Lisboa)입니다. 리스본은 포르투갈의 가장 큰 도시이자 주요 경제, 문화, 정치 중심지입니다.',
  'area': '포르투갈의 면적은 약 92,090 평방 킬로미터(약 35,556 평방 마일)입니다. 이 면적에는 포르투갈 본토뿐만 아니라 아조레스 제도와 마데이라 제도도 포함됩니다.'}]

# RunnablePassthrough

In [12]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
prompt = PromptTemplate.from_template("{num}의 3제곱은?")

In [15]:
chain = prompt|llm|StrOutputParser()

In [16]:
chain.invoke({"num":3})

'3의 3제곱은 \\(3^3\\)으로 계산하면 됩니다. \n\n\\[ 3^3 = 3 \\times 3 \\times 3 = 27 \\]\n\n따라서, 3의 3제곱은 27입니다.'

In [17]:
chain.invoke(3)

'3의 3제곱은 \\(3 \\times 3 \\times 3 = 27\\)입니다.'

In [18]:
RunnablePassthrough().invoke({"num":10})

{'num': 10}

In [20]:
runnable_chain = ({"num":RunnablePassthrough()})|prompt|llm|StrOutputParser()

In [21]:
runnable_chain.invoke(10)

'10의 3제곱은 \\(10^3\\)으로, 이는 10을 세 번 곱한 값을 의미합니다. 계산하면 다음과 같습니다:\n\n\\[ 10 \\times 10 \\times 10 = 1000 \\]\n\n따라서 10의 3제곱은 1000입니다.'

# .assign
- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합친다.

In [22]:
(RunnablePassthrough.assign(new_num= lambda x: x["num"]-2)).invoke({"num":2})

{'num': 2, 'new_num': 0}

# 복합

In [24]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mul=lambda x: x["num"]*10),
    modified = lambda x: x["num"]-3
    
)
runnable.invoke({"num":2})

{'passed': {'num': 2}, 'extra': {'num': 2, 'mul': 20}, 'modified': -1}

In [25]:
chain = ({"num":RunnablePassthrough()}|prompt|llm|StrOutputParser())

In [26]:
chain.invoke(1231)

'1231의 3제곱을 계산해 보겠습니다.\n\n\\[ 1231^3 = 1231 \\times 1231 \\times 1231 \\]\n\n먼저, 1231을 제곱합니다:\n\n\\[ 1231 \\times 1231 = 1,515,361 \\]\n\n그 다음, 이 결과에 다시 1231을 곱합니다:\n\n\\[ 1,515,361 \\times 1231 = 1,865,058,791 \\]\n\n따라서, 1231의 3제곱은 1,865,058,791입니다.'

# RunnableLambda

In [31]:
from langchain_core.runnables import RunnableLambda
from datetime import datetime

In [44]:
def get_today(faker):
    print("input: ",faker)
    return datetime.today().strftime("%b-%d")

In [45]:
get_today(123123)

input:  123123


'Jul-10'

In [46]:
prompt =PromptTemplate.from_template("{today}가 생일인 배우 {n}명을 나열하세요. 생년월일을 표기해 주세요")


In [47]:
chain = (
    {"today":RunnableLambda(get_today),"n":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [48]:
chain.invoke(3)# 3으로 들어감

input:  3


'다음은 7월 10일이 생일인 배우 세 명과 그들의 생년월일입니다:\n\n1. 소피아 베르가라 (Sofía Vergara) - 1972년 7월 10일\n2. 제시카 심슨 (Jessica Simpson) - 1980년 7월 10일\n3. 에이드리언 그레니어 (Adrian Grenier) - 1976년 7월 10일\n\n이 배우들은 각각 다양한 영화와 TV 프로그램에서 활약하며 많은 인기를 얻고 있습니다.'

In [49]:
chain.invoke({"n":3})# RunnablePassthough로 인해 프롬프트에 "n":3이 그대로 들어감

input:  {'n': 3}


'다음은 7월 10일이 생일인 배우 세 명과 그들의 생년월일입니다:\n\n1. 소피아 베르가라 (Sofía Vergara) - 1972년 7월 10일\n2. 제시카 심슨 (Jessica Simpson) - 1980년 7월 10일\n3. 에이드리언 그레니어 (Adrian Grenier) - 1976년 7월 10일'

In [50]:
from operator import itemgetter

In [51]:
chain = (
    {"today":RunnableLambda(get_today),"n":itemgetter("n")}
    |prompt
    |llm
    |StrOutputParser()
)

In [52]:
chain.invoke({"n":3})

input:  {'n': 3}


'다음은 7월 10일이 생일인 배우 세 명과 그들의 생년월일입니다:\n\n1. 소피아 베르가라 (Sofía Vergara) - 1972년 7월 10일\n2. 제시카 심슨 (Jessica Simpson) - 1980년 7월 10일\n3. 에이드리언 그레니어 (Adrian Grenier) - 1976년 7월 10일'

In [53]:
def length_function(text):
    return len(text)
def _mutiple_length_function(text1, text2):
    return len(text1)+len(text2)
def mutiple_length_function(_dict):
    return _mutiple_length_function(_dict['text1'],_dict['text2'])

In [54]:
prompt = PromptTemplate.from_template("{a}+{b}는?")
chain = (
    {
        "a":itemgetter("word1")|RunnableLambda(length_function),
        "b":{"text1": itemgetter("word1"),"text2":itemgetter("word2")}
        |RunnableLambda(mutiple_length_function),
    }
    |prompt
    |llm
    |StrOutputParser()
)

In [55]:
chain.invoke({"word1":"what the","word2":"hack"})

'8 + 12는 20입니다.'

# 혼자 여러 체인 만들어 보기

In [ ]:
# 9시 반 퇴근으로 내일할 예정